In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Step1: upload and clean the dataset

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_excel("/kaggle/input/gym-recommendation/gym recommendation.xlsx")

In [6]:
data.head()

,ID,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
0,1,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, and Lettuc...",Follow a regular exercise schedule. Adhere to ...
1,2,Male,18,1.68,47.5,Yes,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...","Vegetables: (Tomatoes, Garlic, leafy greens, b...",Follow a regular exercise schedule. Adhere to ...
2,3,Male,18,1.68,47.5,No,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and...",Follow a regular exercise schedule. Adhere to ...
3,4,Male,18,1.68,47.5,Yes,Yes,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, yoga, deadlifts, bench presses, and ov...","Light athletic shoes, resistance bands, light ...","Vegetables: (Garlic, Roma Tomatoes, Capers, Gr...",Follow a regular exercise schedule. Adhere to ...
4,5,Male,18,1.68,47.5,No,No,16.83,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, Lettuce); ...",Follow a regular exercise schedule. Adhere to ...


In [8]:
data.columns

Index(['ID', 'Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes',
       'BMI', 'Level', 'Fitness Goal', 'Fitness Type', 'Exercises',
       'Equipment', 'Diet', 'Recommendation'],
      dtype='object')

We don't need ID column, therefore just drop the columsn

In [12]:
data.drop(columns = "ID", inplace = True)

In [13]:
data.sample(3)

,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
9093,Female,41,1.76,45.5,Yes,No,14.69,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...","Vegetables: (Tomatoes, Garlic, leafy greens, b...",Follow a regular exercise schedule. Adhere to ...
9079,Female,40,1.77,86.0,Yes,Yes,27.45,Overweight,Weight Loss,Cardio Fitness,"brisk walking, cycling, swimming, or dancing.","Kettlebell, Dumbbells, Yoga Mat, Treadmill","Vegetables: (Garlic, Mushroom, Green Papper, I...",Follow a regular exercise schedule. Adhere to ...
740,Female,41,1.70,48.0,No,No,16.61,Underweight,Weight Gain,Muscular Fitness,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, Lettuce); ...",Follow a regular exercise schedule. Adhere to ...


In [16]:
data.shape

(14589, 14)

## Step2: Label Encoding

there are several categorical data in this dataset, we have to make them as numerical data for calculation

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['Sex', 'Hypertension', 'Diabetes', 'Fitness Goal', 'Fitness Type']:
    data[col] = le.fit_transform(data[col])

Lets see how the categorical data is converted to numerical values

In [21]:
data.head(3)

,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
0,1,18,1.68,47.5,0,0,16.83,Underweight,0,1,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, and Lettuc...",Follow a regular exercise schedule. Adhere to ...
1,1,18,1.68,47.5,1,0,16.83,Underweight,0,1,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...","Vegetables: (Tomatoes, Garlic, leafy greens, b...",Follow a regular exercise schedule. Adhere to ...
2,1,18,1.68,47.5,0,1,16.83,Underweight,0,1,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and...",Follow a regular exercise schedule. Adhere to ...


the columns Exercises,Equipment, Diet, Recommendation are left to categorical columns only because, they are not used as a input to the model,  they are only used for recommendation part 

## Step3: Normalization

In [22]:
data.describe()

,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Fitness Goal,Fitness Type
count,14589.000000,14589.000000,14589.000000,14589.000000,14589.000000,14589.000000,14589.000000,14589.000000,14589.000000
mean,0.642539,39.554870,1.699851,70.505991,0.462746,0.462746,24.417569,0.519638,0.480362
std,0.479269,13.192655,0.098129,20.030635,0.498627,0.498627,6.765144,0.499631,0.499631
min,0.000000,18.000000,1.300000,32.000000,0.000000,0.000000,9.520000,0.000000,0.000000
25%,0.000000,28.000000,1.640000,55.000000,0.000000,0.000000,18.940000,0.000000,0.000000
50%,1.000000,39.000000,1.680000,70.000000,0.000000,0.000000,25.250000,1.000000,0.000000
75%,1.000000,51.000000,1.770000,86.000000,1.000000,1.000000,29.320000,1.000000,1.000000
max,1.000000,63.000000,2.030000,130.000000,1.000000,1.000000,70.000000,1.000000,1.000000


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[['Age', 'Height', 'Weight','BMI']] = scaler.fit_transform(data[['Age', 'Height', 'Weight','BMI']])

The normalized data looks like this 

In [24]:
data.head(3)

,Sex,Age,Height,Weight,Hypertension,Diabetes,BMI,Level,Fitness Goal,Fitness Type,Exercises,Equipment,Diet,Recommendation
0,1,-1.63391,-0.202298,-1.14858,0,0,-1.121606,Underweight,0,1,"Squats, deadlifts, bench presses, and overhead...",Dumbbells and barbells,"Vegetables: (Carrots, Sweet Potato, and Lettuc...",Follow a regular exercise schedule. Adhere to ...
1,1,-1.63391,-0.202298,-1.14858,1,0,-1.121606,Underweight,0,1,"Squats, deadlifts, bench presses, and overhead...","Light athletic shoes, resistance bands, and li...","Vegetables: (Tomatoes, Garlic, leafy greens, b...",Follow a regular exercise schedule. Adhere to ...
2,1,-1.63391,-0.202298,-1.14858,0,1,-1.121606,Underweight,0,1,"Squats, yoga, deadlifts, bench presses, and ov...","Dumbbells, barbells and Blood glucose monitor","Vegetables: (Garlic, Roma Tomatoes, Capers and...",Follow a regular exercise schedule. Adhere to ...


## Step4: Recommendation, Feedback, Evaluation

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_recommendation(top_n = 3):
    print("Please enter your details for personalised workout and diet plan for recommendation ")
    user_input = {
        'Sex': int(input("Enter Sex (Male : 1/Female : 0): ")),
        'Age': float(input("Enter Age: ")),
        'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
        'Weight': float(input("Enter Weight in kg: ")),
        'Hypertension': int(input("Do you have Hypertension (Yes : 1/No : 0): ")),
        'Diabetes': int(input("Do you have Diabetes (Yes : 1/No : 0): ")),
        'BMI': float(input("Enter BMI: ")),
        'Level': int(input("Enter Level (Underweight : 3, Normal : 0, Overweight : 2, Obese : 1): ")),
        'Fitness Goal': int(input("Enter Fitness Goal (Weight Gain : 0, Weight Loss : 1): ")),
        'Fitness Type': int(input("Enter Fitness Type (Muscular Fitness : 1, Cardio Fitness : 0): "))
    }

    # Normalise the inputs
    num_features = ['Age', 'Height', 'Weight','BMI']
    user_df = pd.DataFrame(user_input, columns = num_features)
    
    user_df[num_features] = scaler.transform(user_df[num_features])
    
    user_input.update(user_df.iloc[0].to_dict())
    user_df.pd.DataFrame([user_input])
    
    # Calculate similarity scores
    user_features = data[['Sex', 'Age', 'Height', 'Weight', 'Hypertension','Diabetes', 'BMI', 'Level', 'Fitness Goal', 'Fitness Type']]
    
    sim_scores = cosine_similarity(user_features, user_df).flatten()
    
    # Retrive Top similar users and get the first recommendation
    similar_user_indices = sim_scores.argsort()[-5:][::-1]
    similar_users = data.iloc[similar_user_indices]
    
    recommendation_1 = similar_users[['Exercises','Diet','Equipment']].mod().iloc[0]
    
    # Stimulate 2 additional recommendation y modifying user input slightly
    stimulated_recommendation = []
    for _ in range(2):
        modfied_input = user_input.copy()
        # Randomly adjusting columns age, weight, BMI with larger variation
        modified_input['Age'] += random.randint(-5, 5)
        modified_input['Wright'] += random.randint(-5, 5)
        modified_input['BMI'] += random.randint(-1, 1)
        
        # Normalise the modified inputs
        modified_user_df = pd.DataFrame([modified_input], columns=num_features)
        modified_user_df[num_features] = scaler.transform(modified_user_df[num_features]) 
        modfied_input.update(modified_user_df.iloc[0].to_dict())
        
        # Calculating similarity scores for modified Input
        modified_sim_scores = cosine_similarity(user_features, pd.DataFrame([modified_input])).flatten()
        modified_sim_user_indices = modified_sim_scores.argsort()[-5:][::-1]
        modified_sim_users = data.iloc[modified_sim_user_indices]
    
        recommendation = modified_sim_users[['Exercises','Diet','Equipment']].mode().iloc[0]
    
        # Check if the recommendation is already in stimulated rec
        if not any(rec['Exercises'] == recommendation['Exercises'] and rec['Diet'] == recommendation['Diet'] and rec['Equipment'] == recommendation['Equipment'] for rec in simulated_recommendations):
            simulated_recommendations.append(recommendation)
    
     # Display all recommendations
        print("\nRecommended Workout and Diet Plans based on your input:")
        print("\nRecommendation 1 (Exact match):")
        print("EXERCISES:", recommendation_1['Exercises'])
        print("EQUIPMENTS:", recommendation_1['Equipment'])
        print("DIET:", recommendation_1['Diet'])
        
    for idx, rec in enumerate(simulated_recommendations, start=2):
            print(f"\nRecommendation {idx} (Slight variation):")
            print("EXERCISES:", rec['Exercises'])
            print("EQUIPMENTS:", rec['Equipment'])
            print("DIET:", rec['Diet'])
    
    # Collecet feedback for each recommendation
    feedback_matrix = []
    for i in range(len(stimulated_recommendation0 + 1)):
        feedback = int(input("Was recommendation i+1 relevant? (if Yes: 1, No: 0): "))
        feedback_matrix.append(feedback)
    
    # Calculate mean reciprocle rank (MRR)
    relevant_indices = [i + 1 for i, feedback in enumerate(feedback_matrix) if feedback == 1]
    if relevant_idices:
        mrr = np.mean([1/rank for rank in relevant_indices])
    else:
        mrr = 0.0
    print(f"\n Mean Reciprocal Rank(MRR): {mrr:.2f}")
    return [recommendation_1] + stimulated_recommendations

# Get the recommendation
recommendations = get_recommendation(top_n = 3)